In [1]:
!pip install fastembed > /dev/null 2>&1

In [2]:
from fastembed import TextEmbedding

In [3]:
queries: list[str] = [
    "I just discovered the course. Can I join now?",
]

In [4]:
def embeddingsFor(docs, model_name="jinaai/jina-embeddings-v2-small-en"):
    model = TextEmbedding(model_name=model_name)
    return list(model.embed(docs))

In [5]:
embeddings = embeddingsFor(queries)

In [6]:
q = embeddings[0]

In [7]:
print(f"Emebedding vector Length: {len(q)}")

Emebedding vector Length: 512


In [8]:
print(f"Q1: {min(embeddings[0])}")

Q1: -0.11726373022974847


In [9]:
doc = 'Can I still join the course after the start date?'

In [10]:
print(f"Q2: {embeddingsFor(doc)[0].dot(q)}")



Q2: 0.9008529113860175


In [11]:
documents: list[str] = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [12]:
documentVectors = embeddingsFor(list(map(lambda document: document['text'], documents)))

In [13]:
len(documentVectors[0])

512

In [14]:
documentVectors[0].dot(q)

np.float64(0.762968452623467)

In [15]:
cosines = list(map(lambda v:  v.dot(q), documentVectors));
cosines

[np.float64(0.762968452623467),
 np.float64(0.8182378205962001),
 np.float64(0.8085397376859877),
 np.float64(0.713307864606494),
 np.float64(0.7304499263004142)]

In [16]:
index, _ = max(enumerate(cosines), key=lambda pair: pair[1]);
print(f"Q3: {index}")

Q3: 1


In [17]:
import numpy as np

In [18]:
def entryWithMaxValue(values):
    index, _ = max(enumerate(values), key=lambda pair: pair[1])
    return index

In [19]:
def entryWithMaxSimilarity(documents, q, model_name=None):
    embeddings = None
    if model_name is None:
          embeddings = embeddingsFor(documents)
    else:
          embeddings = embeddingsFor(documents, model_name)
    matrix = np.array(embeddings)
    cosines = matrix.dot(q)
    return entryWithMaxValue(cosines)


In [20]:
matrix = np.array(documentVectors)
cosines1 = matrix.dot(q)
cosines1

array([0.76296845, 0.81823782, 0.80853974, 0.71330786, 0.73044993])

In [21]:
entryWithMaxValue(cosines1)

1

In [22]:
entryWithMaxSimilarity(list(map(lambda document: document['text'], documents)), q)

1

In [23]:
Q4 = entryWithMaxSimilarity(list(map(lambda document: document['question'] + ' ' + document['text'], documents)), q)
print(f"Q4: {Q4}")

Q4: 0


In [24]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

len(documents)
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [25]:
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [26]:
embeddings: list[np.ndarray] = list(
    embedding_model.passage_embed(list(map(lambda document: document['question'] + ' ' + document['text'], documents)))
)
query = queries[0]
query_embedding = list(embedding_model.query_embed(query))[0]
plain_query_embedding = list(embedding_model.embed(query))[0]

scores = np.dot(embeddings, query_embedding)
print(f"Q6: {max(scores)}")

Q6: 0.8620737698280204
